In [ ]:
%pip install python-dotenv pandas chardet

In [8]:
import_file = "George-CSV-Export/2025-01-01_2025-12-31.csv"

In [ ]:
import chardet

# Open the CSV file in binary mode
with open(import_file, 'rb') as file:
    raw_data = file.read()

# Detect the encoding
result = chardet.detect(raw_data)

csv_encoding = result['encoding']

# Print the detected encoding
print(f"Detected encoding: {csv_encoding}")

In [ ]:
import pandas as pd

# Load CSV file into DataFrame
df = pd.read_csv(import_file, encoding=csv_encoding, parse_dates=[
    "Buchungsdatum", 
    "Valutadatum"
    ], dayfirst=True, keep_default_na=False)


# apply data type fixes
df['Highlight'] = df['Highlight'].astype(bool)

# input number format: "10.123,02"
# output: 100123.02
df['Betrag'] = df['Betrag'].apply( lambda x: float(x.replace(".", "").replace(",", ".")))

# Display the first few rows of the DataFrame
df.head()

In [ ]:
df.dtypes

In [12]:


import psycopg
import os
import pytz

os.environ['PGSERVICEFILE'] = "/home/martin/Workspace/Energiegemeinschaft/notebooks/.pg_service.conf"
os.environ['PGPASSFILE'] = "/home/martin/Workspace/Energiegemeinschaft/notebooks/.pgpass"

with psycopg.connect(service='eeg-middleware') as conn:
    with conn.cursor() as cur:
        
        
        sql = """
        INSERT INTO accounting_booking (
            account_name,
            iban,
            booking_date,
            partner_name,
            partner_iban,
            bic_swift,
            partner_account_number,
            bank_code,
            amount,
            currency,
            booking_details,
            booking_reference,  -- unique reference
            note,
            highlight,
            value_date,
            virtual_card_number,
            paid_with,
            app,
            payment_reference,
            mandate_id,
            creditor_id
        )
        VALUES (
            %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s
        )
        ON CONFLICT (booking_date, partner_iban, amount, currency, booking_details, value_date, mandate_id, creditor_id, payment_reference, booking_reference)
        DO NOTHING
        """


        for index, row in df.iterrows():
            values = (
                row['Eigener Kontoname'],  # account_name
                row['Eigene IBAN'],  # iban
                row['Buchungsdatum'],  # booking_date
                row['Partnername'],  # partner_name
                row['Partner IBAN'],  # partner_iban
                row['BIC/SWIFT'],  # bic_swift
                row['Partner Kontonummer'],  # partner_account_number
                row['Bankleitzahl'],  # bank_code
                row['Betrag'],  # amount
                row['Währung'],  # currency
                row['Buchungs-Details'],  # booking_details
                row['Buchungsreferenz'],  # booking_reference (unique key)
                row['Notiz'],  # note
                row['Highlight'],  # highlight
                row['Valutadatum'],  # value_date
                row['Virtuelle Kartennummer'],  # virtual_card_number
                row['Bezahlt mit'],  # paid_with
                row['App'],  # app
                row['Zahlungsreferenz'],  # payment_reference
                row['Mandats ID'],  # mandate_id
                row['Creditor ID']  # creditor_id
            )
            # Execute the SQL query for each row
            cur.execute(sql, values)
        
        conn.commit()


